### Week 1: Data Science & Data Preparation

#### Setup Environment and Load Libraries

In [1]:
import pandas as pd
import numpy as np

#### Load the excel files

In [2]:
flights_df = pd.read_excel("Airlines.xlsx")
airports_df = pd.read_excel("airports.xlsx")
runways_df = pd.read_excel("runways.xlsx")

#### Aggregate Runway Data

In [3]:
runway_agg_df = runways_df.groupby('airport_ident').agg(
    number_of_runways=('id', 'count'),
    avg_runway_length_ft=('length_ft', 'mean'),
    avg_runway_width_ft=('width_ft', 'mean')
).reset_index()

#### Merge Flights with Source and Destination Airport Data

In [4]:
merged_df = pd.merge(
    flights_df,
    airports_df,
    left_on='AirportFrom',
    right_on='iata_code',
    how='left',
    suffixes=('', '_from')
)

merged_df = pd.merge(
    merged_df,
    airports_df,
    left_on='AirportTo',
    right_on='iata_code',
    how='left',
    suffixes=('', '_to')
)

#### Merge the Aggregated Runway Data

In [5]:
merged_df = pd.merge(
    merged_df,
    runway_agg_df,
    left_on='ident',
    right_on='airport_ident',
    how='left',
    suffixes=('', '_from_runway')
)

merged_df = pd.merge(
    merged_df,
    runway_agg_df,
    left_on='ident_to', 
    right_on='airport_ident',
    how='left',
    suffixes=('_from_runway', '_to_runway')
)

### Task 1b & 1c: Web Scraping

#### Scrape Airline Founding Data

In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find('table', {'class': 'wikitable'}) 

#### Scrape Airport Hub Data

In [7]:
url_hubs = "https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States"

### Task 2: Missing Value Treatment

#### Identify Missing Values

In [8]:
print(merged_df.isnull().sum())

id                                       0
Airline                                  0
Flight                                   0
AirportFrom                              0
AirportTo                                0
DayOfWeek                                0
Time                                     0
Length                                   0
Delay                                    0
id_from                                 31
ident                                   31
type                                    31
name                                    31
latitude_deg                            31
longitude_deg                           31
elevation_ft                            31
continent                           509779
iso_country                             31
iso_region                              31
municipality                            31
scheduled_service                       31
gps_code                                31
iata_code                               31
local_code 

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml

print("STEP 1: Loading initial Excel files...")
try:
    flights_df = pd.read_excel("Airlines.xlsx")
    airports_df = pd.read_excel("airports.xlsx")
    runways_df = pd.read_excel("runways.xlsx")
    print("Files loaded successfully.")
except FileNotFoundError as e:
    print(f"FATAL ERROR: Could not find a file. Make sure they are in the correct folder. Details: {e}")
    raise

print("\n STEP 2: Aggregating runway data...")
runway_agg_df = runways_df.groupby('airport_ident').agg(
    number_of_runways=('id', 'count'),
    avg_runway_length_ft=('length_ft', 'mean'),
    avg_runway_width_ft=('width_ft', 'mean')
).reset_index()
print("Runway data aggregated.")

print("\n STEP 3: Merging flights with airport and runway data...")
merged_df = pd.merge(flights_df, airports_df, left_on='AirportFrom', right_on='iata_code', how='left')
merged_df = pd.merge(merged_df, airports_df, left_on='AirportTo', right_on='iata_code', how='left', suffixes=('_from', '_to'))
merged_df = pd.merge(merged_df, runway_agg_df, left_on='ident_from', right_on='airport_ident', how='left', suffixes=('', '_from_runway'))
merged_df = pd.merge(merged_df, runway_agg_df, left_on='ident_to', right_on='airport_ident', how='left', suffixes=('_from_runway', '_to_runway'))
print(" All initial merges complete.")

print("\n STEP 4: Scraping, cleaning, and merging hub data...")
try:
    url_hubs = "https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States"
    tables = pd.read_html(url_hubs)
    hub_df = tables[0]
    
    if isinstance(hub_df.columns, pd.MultiIndex):
        hub_df.columns = hub_df.columns.get_level_values(1)
        
    hub_df = hub_df[['Airport', 'IATA']]
    hub_df['Hub_Size'] = 'Small/Non-Hub'
    hub_df.loc[hub_df.index < 30, 'Hub_Size'] = 'Large Hub'
    hub_df.loc[(hub_df.index >= 30) & (hub_df.index < 70), 'Hub_Size'] = 'Medium Hub'
    
    merged_df = pd.merge(merged_df, hub_df[['IATA', 'Hub_Size']], left_on='AirportFrom', right_on='IATA', how='left')
    merged_df.rename(columns={'Hub_Size': 'Hub_Size_from'}, inplace=True)
    # Merge for destination
    merged_df = pd.merge(merged_df, hub_df[['IATA', 'Hub_Size']], left_on='AirportTo', right_on='IATA', how='left')
    merged_df.rename(columns={'Hub_Size': 'Hub_Size_to'}, inplace=True)
    
    merged_df.drop(columns=['IATA_x', 'IATA_y'], inplace=True, errors='ignore')
    print(" Hub data scraped and merged successfully.")
except Exception as e:
    print(f" WARNING: Could not scrape or merge hub data. Details: {e}")
    print("   Creating placeholder columns for Hub_Size to allow script to continue.")
    merged_df['Hub_Size_from'] = 'Unknown'
    merged_df['Hub_Size_to'] = 'Unknown'

print("\n--- Verifying Columns Exist Before Imputation ---")
print(merged_df.columns.tolist())
print("-------------------------------------------------")

print("\n STEP 5: Starting final data imputation...")

print("   Imputing missing elevation data...")
merged_df['elevation_ft_from'].fillna(merged_df['elevation_ft_from'].median(), inplace=True)
merged_df['elevation_ft_to'].fillna(merged_df['elevation_ft_to'].median(), inplace=True)
print(" Elevation data imputed.")

print("   Imputing missing runway data...")
merged_df['avg_runway_length_ft_from_runway'].fillna(merged_df['avg_runway_length_ft_from_runway'].median(), inplace=True)
merged_df['number_of_runways_from_runway'].fillna(0, inplace=True)
merged_df['avg_runway_length_ft_to_runway'].fillna(merged_df['avg_runway_length_ft_to_runway'].median(), inplace=True)
merged_df['number_of_runways_to_runway'].fillna(0, inplace=True)
print("Runway data imputed.")

print("   Imputing missing hub status...")
merged_df['Hub_Size_from'].fillna('Small/Non-Hub', inplace=True)
merged_df['Hub_Size_to'].fillna('Small/Non-Hub', inplace=True)
print("Hub status imputed.")

print("\n\n ALL DATA PREPARATION COMPLETE. You can now proceed to visualization and modeling.")

STEP 1: Loading initial Excel files...
Files loaded successfully.

 STEP 2: Aggregating runway data...
Runway data aggregated.

 STEP 3: Merging flights with airport and runway data...
 All initial merges complete.

 STEP 4: Scraping, cleaning, and merging hub data...
   Creating placeholder columns for Hub_Size to allow script to continue.

--- Verifying Columns Exist Before Imputation ---
['id_x', 'Airline', 'Flight', 'AirportFrom', 'AirportTo', 'DayOfWeek', 'Time', 'Length', 'Delay', 'id_y', 'ident_from', 'type_from', 'name_from', 'latitude_deg_from', 'longitude_deg_from', 'elevation_ft_from', 'continent_from', 'iso_country_from', 'iso_region_from', 'municipality_from', 'scheduled_service_from', 'gps_code_from', 'iata_code_from', 'local_code_from', 'home_link_from', 'wikipedia_link_from', 'keywords_from', 'id', 'ident_to', 'type_to', 'name_to', 'latitude_deg_to', 'longitude_deg_to', 'elevation_ft_to', 'continent_to', 'iso_country_to', 'iso_region_to', 'municipality_to', 'scheduled_s

C:\Users\HP\AppData\Local\Temp\ipykernel_2240\2430011939.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['elevation_ft_from'].fillna(merged_df['elevation_ft_from'].median(), inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_2240\2430011939.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

### Machine Learning 

#### Import Libraries and Define Features

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

print(" All machine learning libraries imported successfully.")

features = [
    'Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek', 'Time', 'Length',
    'elevation_ft_from', 'number_of_runways_from_runway', 'Hub_Size_from'
]

target = 'Delay'

X = merged_df[features]
y = merged_df[target]

print(f"\nFeatures (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")

 All machine learning libraries imported successfully.

Features (X) shape: (518556, 9)
Target (y) shape: (518556,)


#### Split Data into Training and Testing Sets

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      
    random_state=42,   
    stratify=y          
)

print("Data split into training and testing sets:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

Data split into training and testing sets:
X_train shape: (414844, 9)
X_test shape: (103712, 9)


####  Create the Preprocessing Pipeline

In [12]:
categorical_features = ['Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek', 'Hub_Size_from']
numerical_features = ['Time', 'Length', 'elevation_ft_from', 'number_of_runways_from_runway']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

print("Preprocessing pipeline created successfully.")

Preprocessing pipeline created successfully.


####  Build, Train, and Evaluate Model 1 (Logistic Regression)

In [13]:
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log_loss', random_state=42, max_iter=1000, tol=1e-3))
])

print("Training the Logistic Regression model...")
lr_pipeline.fit(X_train, y_train)
print("Training complete.")

y_pred_lr = lr_pipeline.predict(X_test)

print("\n--- Logistic Regression Classification Report ---")
print(classification_report(y_test, y_pred_lr))
print("-------------------------------------------------")

Training the Logistic Regression model...
Training complete.

--- Logistic Regression Classification Report ---
              precision    recall  f1-score   support

           0       0.65      0.75      0.70     56914
           1       0.62      0.51      0.56     46798

    accuracy                           0.64    103712
   macro avg       0.64      0.63      0.63    103712
weighted avg       0.64      0.64      0.63    103712

-------------------------------------------------


#### Build, Train, and Evaluate Model 2 (Decision Tree with Pruning)

In [14]:
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

param_grid = {
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_leaf': [10, 20, 50]
}

print("\nSearching for the best Decision Tree parameters (this may take a moment)...")
grid_search_dt = GridSearchCV(dt_pipeline, param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)
print("Search complete.")
print(f"Best parameters found: {grid_search_dt.best_params_}")

best_dt_model = grid_search_dt.best_estimator_
y_pred_dt = best_dt_model.predict(X_test)

print("\n--- Tuned Decision Tree Classification Report ---")
print(classification_report(y_test, y_pred_dt))
print("-------------------------------------------------")


Searching for the best Decision Tree parameters (this may take a moment)...
Search complete.
Best parameters found: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 50}

--- Tuned Decision Tree Classification Report ---
              precision    recall  f1-score   support

           0       0.65      0.80      0.72     56914
           1       0.66      0.47      0.55     46798

    accuracy                           0.65    103712
   macro avg       0.65      0.64      0.63    103712
weighted avg       0.65      0.65      0.64    103712

-------------------------------------------------


#### Build, Train, and Evaluate Model 3 (Gradient Boosting)

In [15]:
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
])

print("\nTraining the Gradient Boosting model (this can be slow)...")
gb_pipeline.fit(X_train, y_train)
print("Training complete.")

y_pred_gb = gb_pipeline.predict(X_test)

print("\n--- Gradient Boosting Classification Report ---")
print(classification_report(y_test, y_pred_gb))
print("-------------------------------------------------")


Training the Gradient Boosting model (this can be slow)...
Training complete.

--- Gradient Boosting Classification Report ---
              precision    recall  f1-score   support

           0       0.64      0.84      0.72     56914
           1       0.68      0.42      0.52     46798

    accuracy                           0.65    103712
   macro avg       0.66      0.63      0.62    103712
weighted avg       0.66      0.65      0.63    103712

-------------------------------------------------


#### Final Comparison

In [16]:
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_dt = accuracy_score(y_test, y_pred_dt)
acc_gb = accuracy_score(y_test, y_pred_gb)

summary = pd.DataFrame({
    'Model': ['Logistic Regression', 'Tuned Decision Tree', 'Gradient Boosting'],
    'Test Accuracy': [acc_lr, acc_dt, acc_gb]
})

print("\n--- Final Model Performance Summary ---")
print(summary.sort_values(by='Test Accuracy', ascending=False))
print("---------------------------------------")


--- Final Model Performance Summary ---
                 Model  Test Accuracy
1  Tuned Decision Tree       0.651101
2    Gradient Boosting       0.648151
0  Logistic Regression       0.640302
---------------------------------------


In [17]:
output_filename = "final_airline_delay_dataset.csv"

print(f" Preparing to extract the final dataset...")
print(f"   The final DataFrame has {merged_df.shape[0]} rows and {merged_df.shape[1]} columns.")

try:
    merged_df.to_csv(output_filename, index=False)
    
    print(f"\n Success! The final dataset has been extracted and saved as '{output_filename}'")
    print("   You can now find this file in the same directory as your Jupyter Notebook.")
    
except Exception as e:
    print(f"\n An error occurred while trying to save the file: {e}")



 Preparing to extract the final dataset...
   The final DataFrame has 518556 rows and 55 columns.

 Success! The final dataset has been extracted and saved as 'final_airline_delay_dataset.csv'
   You can now find this file in the same directory as your Jupyter Notebook.


In [18]:
final_dataframe_to_export = merged_df

output_filename = "final_airline_delay_dataset(1).csv"

print(f" Re-exporting the final dataset to fix the header issue...")

final_dataframe_to_export.to_csv(output_filename, index=False)

print(f"\n Success! The file '{output_filename}' has been re-saved correctly without the index column.")


 Re-exporting the final dataset to fix the header issue...

 Success! The file 'final_airline_delay_dataset(1).csv' has been re-saved correctly without the index column.


In [19]:
import pandas as pd

filename = "final_airline_delay_dataset(1).csv"
print(f"Inspecting file: {filename}\n")

try:
    df_to_inspect = pd.read_csv(filename)

    csv_columns = df_to_inspect.columns.tolist()
    
    print("--- Columns found in the CSV file ---")
    print(csv_columns)
    print(f"\nTotal number of columns in CSV: {len(csv_columns)}")
    
    unnamed_cols = [col for col in csv_columns if 'Unnamed' in col]
    
    if unnamed_cols:
        print(f"\n CRITICAL ISSUE FOUND: The CSV contains unnamed columns: {unnamed_cols}")
        print("   This is the most likely cause of the error. These must be removed.")
    else:
        print("\n Good news: No columns are named 'Unnamed'. The headers seem clean.")

except Exception as e:
    print(f" An error occurred while trying to read the CSV file: {e}")


Inspecting file: final_airline_delay_dataset(1).csv

--- Columns found in the CSV file ---
['id_x', 'Airline', 'Flight', 'AirportFrom', 'AirportTo', 'DayOfWeek', 'Time', 'Length', 'Delay', 'id_y', 'ident_from', 'type_from', 'name_from', 'latitude_deg_from', 'longitude_deg_from', 'elevation_ft_from', 'continent_from', 'iso_country_from', 'iso_region_from', 'municipality_from', 'scheduled_service_from', 'gps_code_from', 'iata_code_from', 'local_code_from', 'home_link_from', 'wikipedia_link_from', 'keywords_from', 'id', 'ident_to', 'type_to', 'name_to', 'latitude_deg_to', 'longitude_deg_to', 'elevation_ft_to', 'continent_to', 'iso_country_to', 'iso_region_to', 'municipality_to', 'scheduled_service_to', 'gps_code_to', 'iata_code_to', 'local_code_to', 'home_link_to', 'wikipedia_link_to', 'keywords_to', 'airport_ident_from_runway', 'number_of_runways_from_runway', 'avg_runway_length_ft_from_runway', 'avg_runway_width_ft_from_runway', 'airport_ident_to_runway', 'number_of_runways_to_runway', 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.mysql import LONGTEXT
from sqlalchemy.exc import SQLAlchemyError

try:
    db_user = 'root'
    db_password = 'Pu*TM3vq7W9r' 
    db_host = 'localhost'
    db_name = 'airlines_db'

    connection_string = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"
    engine = create_engine(connection_string)
    
    print(" Database connection engine created successfully.")

except Exception as e:
    print(f" Error creating database connection: {e}")
    raise

filename = "final_airline_delay_dataset(1).csv"
print(f"\nLoading data from {filename}...")
df_to_load = pd.read_csv(filename)

print("\nCreating explicit dtype mapping for SQL table...")
all_columns = df_to_load.columns
dtype_mapping = {}
for col in all_columns:
    if df_to_load[col].dtype == 'object':
        dtype_mapping[col] = types.VARCHAR(length=255)
    elif 'int' in str(df_to_load[col].dtype):
        dtype_mapping[col] = types.BIGINT
    elif 'float' in str(df_to_load[col].dtype):
        dtype_mapping[col] = types.DECIMAL(precision=10, scale=2)

long_text_columns = [
    'keywords_from', 'keywords_to', 'name_from', 'name_to', 
    'home_link_from', 'home_link_to', 'wikipedia_link_from', 'wikipedia_link_to'
]
for col in long_text_columns:
    if col in dtype_mapping:
        dtype_mapping[col] = LONGTEXT
print(" Dtype mapping created successfully.")


chunk_size = 10000 
table_name = 'final_data'
start_row = 0

try:
    print(f"\nCreating a new, empty table named '{table_name}'...")
    df_to_load.head(0).to_sql(
        name=table_name, 
        con=engine, 
        if_exists='replace', 
        index=False,
        dtype=dtype_mapping
    )
    print(" Empty table created successfully.")

    print("\nStarting to load data in chunks...")
    for i in range(0, len(df_to_load), chunk_size):
        start_row = i
        end_row = i + chunk_size
        chunk = df_to_load.iloc[start_row:end_row]
        
        print(f"  - Loading rows {start_row} to {end_row-1}...")
        
        chunk.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False
        )
    
    print("\n SUCCESS! All chunks loaded successfully.")

except SQLAlchemyError as e:
    print(f"\n A SQLAlchemy error occurred while processing the chunk starting at row {start_row}.")
    print(f"      The problematic data is likely between row {start_row} and {start_row + chunk_size - 1} in your CSV file.")
    print(f"      Original Error: {e}")
    
except Exception as e:
    print(f"\n An unexpected error occurred: {e}")

finally:
    engine.dispose()
    print("\nDatabase connection engine disposed.")